<a href="https://colab.research.google.com/github/softmurata/colab_notebooks/blob/main/pix2pix/controlnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title RefDocs
# https://huggingface.co/lllyasviel/ControlNet
# https://github.com/lllyasviel/ControlNet
# https://self-development.info/%e5%a5%bd%e3%81%8d%e3%81%aa%e3%83%a2%e3%83%87%e3%83%ab%e3%81%a7controlnet%e3%81%ab%e3%82%88%e3%82%8b%e7%94%bb%e5%83%8f%e7%94%9f%e6%88%90%e3%82%92%e8%a1%8c%e3%81%86%e6%96%b9%e6%b3%95/

In [ ]:
!git clone https://github.com/lllyasviel/ControlNet.git

Cloning into 'ControlNet'...
remote: Enumerating objects: 820, done.
remote: Counting objects: 100% (155/155), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 820 (delta 105), reused 87 (delta 83), pack-reused 665
Receiving objects: 100% (820/820), 84.33 MiB | 33.10 MiB/s, done.
Resolving deltas: 100% (250/250), done.


In [ ]:
# requirements.txt
"""
gradio
albumentations
opencv-contrib-python
imageio
imageio-ffmpeg
pytorch-lightning
omegaconf
test-tube
streamlit
einops
transformers
webdataset
kornia
open_clip_torch
invisible-watermark
streamlit-drawable-canvas
torchmetrics
timm
addict
yapf
prettytable
"""

In [ ]:
%cd ControlNet
!pip install -r requirements.txt

In [ ]:
!pip install xformers triton

In [ ]:
#@title canny
!wget https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_canny.pth -P models
# !wget https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_openpose.pth -P models
# !wget https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_depth.pth -P models
# !wget https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_mlsd.pth -P models

In [ ]:
!python gradio_canny2image.py

In [ ]:
#@title pose
!wget https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_openpose.pth -P models

In [ ]:
!wget https://huggingface.co/lllyasviel/ControlNet/resolve/main/annotator/ckpts/body_pose_model.pth -P annotator/ckpts

In [ ]:
!wget https://huggingface.co/lllyasviel/ControlNet/resolve/main/annotator/ckpts/hand_pose_model.pth -P annotator/ckpts

In [ ]:
# Please change gradio_pose2image.py -> block.launch(share=True)
!python gradio_pose2image.py

In [ ]:
#@title mlsd
!wget https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_mlsd.pth -P models

In [ ]:
!wget https://huggingface.co/lllyasviel/ControlNet/resolve/main/annotator/ckpts/mlsd_large_512_fp32.pth -P annotator/ckpts

In [ ]:
!python gradio_hough2image.py

In [ ]:
#@title Advanced 
# Anything-full -> https://kurokumasoft.com/2023/01/22/stable-diffusion-anything/
!wget https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/anything-v3-full.safetensors -P models

In [ ]:
!wget https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned.ckpt -P models

In [ ]:
!nvidia-smi

In [ ]:
# cannot transfer because of out of memory
!python tool_transfer_control.py

2023-02-15 01:13:06.248756: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-15 01:13:10.105106: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-15 01:13:10.105784: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-15 01:13:10.105809: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

In [1]:
#@title pix2pix training
# get training dataset
!wget https://huggingface.co/lllyasviel/ControlNet/resolve/main/training/fill50k.zip

--2023-02-17 02:47:32--  https://huggingface.co/lllyasviel/ControlNet/resolve/main/training/fill50k.zip
Resolving huggingface.co (huggingface.co)... 54.235.118.239, 3.231.67.228, 2600:1f18:147f:e800:671:b733:ecf3:a585, ...
Connecting to huggingface.co (huggingface.co)|54.235.118.239|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/f6/65/f665e553e8fd16131981a96c629e785939f08080bb3cbf8d2b7f5ddbd9e6100a/8d87956c454581c921fab2127bf0f89add40add3fc63933a1a4b6473366cfd52?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27fill50k.zip%3B+filename%3D%22fill50k.zip%22%3B&Expires=1676861253&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2Y2LzY1L2Y2NjVlNTUzZThmZDE2MTMxOTgxYTk2YzYyOWU3ODU5MzlmMDgwODBiYjNjYmY4ZDJiN2Y1ZGRiZDllNjEwMGEvOGQ4Nzk1NmM0NTQ1ODFjOTIxZmFiMjEyN2JmMGY4OWFkZDQwYWRkM2ZjNjM5MzNhMWE0YjY0NzMzNjZjZmQ1Mj9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoiLCJDb25kaXRpb24iO

In [2]:
!git clone https://github.com/lllyasviel/ControlNet.git

Cloning into 'ControlNet'...
remote: Enumerating objects: 880, done.
remote: Total 880 (delta 0), reused 0 (delta 0), pack-reused 880
Receiving objects: 100% (880/880), 84.34 MiB | 27.10 MiB/s, done.
Resolving deltas: 100% (276/276), done.


In [3]:
!unzip fill50k.zip

ストリーミング出力は最後の 5000 行に切り捨てられました。
  inflating: fill50k/target/5499.png  
  inflating: fill50k/target/55.png   
  inflating: fill50k/target/550.png  
  inflating: fill50k/target/5500.png  
  inflating: fill50k/target/5501.png  
  inflating: fill50k/target/5502.png  
  inflating: fill50k/target/5503.png  
  inflating: fill50k/target/5504.png  
  inflating: fill50k/target/5505.png  
  inflating: fill50k/target/5506.png  
  inflating: fill50k/target/5507.png  
  inflating: fill50k/target/5508.png  
  inflating: fill50k/target/5509.png  
  inflating: fill50k/target/551.png  
  inflating: fill50k/target/5510.png  
  inflating: fill50k/target/5511.png  
  inflating: fill50k/target/5512.png  
  inflating: fill50k/target/5513.png  
  inflating: fill50k/target/5514.png  
  inflating: fill50k/target/5515.png  
  inflating: fill50k/target/5516.png  
  inflating: fill50k/target/5517.png  
  inflating: fill50k/target/5518.png  
  inflating: fill50k/target/5519.png  
  inflating: fill50k/target/552.png

In [4]:
import os
import subprocess
import json

number = 100

os.makedirs(f"/content/fill{number}/source", exist_ok=True)
os.makedirs(f"/content/fill{number}/target", exist_ok=True)

for i in range(number):
  cmd = f"cp /content/fill50k/source/{i}.png /content/fill{number}/source/{i}.png"
  subprocess.call(cmd, shell=True)
  cmd = f"cp /content/fill50k/target/{i}.png /content/fill{number}/target/{i}.png"
  subprocess.call(cmd, shell=True)

data = []
with open('/content/fill50k/prompt.json', 'rt') as f:
  idx = 0
  for line in f:
    if idx == number:
      break
    data.append(json.loads(line))
    idx += 1

with open(f"/content/fill{number}/prompt.json", "wt") as f:
  for line in data:
    f.write(json.dumps(line) + "\n")


In [ ]:
# create my dataset
import json
import cv2
import numpy as np

from torch.utils.data import Dataset


class MyDataset(Dataset):
    def __init__(self, number):
        self.number = number
        self.data = []
        with open(f'/content/fill{number}/prompt.json', 'rt') as f:
            for line in f:
                self.data.append(json.loads(line))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]

        source_filename = item['source']
        target_filename = item['target']
        prompt = item['prompt']

        

        source = cv2.imread(f'/content/fill{self.number}/' + source_filename)
        target = cv2.imread(f'/content/fill{self.number}/' + target_filename)

        # Do not forget that OpenCV read images in BGR order.
        source = cv2.cvtColor(source, cv2.COLOR_BGR2RGB)
        target = cv2.cvtColor(target, cv2.COLOR_BGR2RGB)

        # Normalize source images to [0, 1].
        source = source.astype(np.float32) / 255.0

        # Normalize target images to [-1, 1].
        target = (target.astype(np.float32) / 127.5) - 1.0

        return dict(jpg=target, txt=prompt, hint=source)

In [ ]:
dataset = MyDataset(number)
print(len(dataset))

item = dataset[12]
jpg = item['jpg']
txt = item['txt']
hint = item['hint']
print(txt)
print(jpg.shape)
print(hint.shape)

In [ ]:
# get stable diffusion 2 weight
!mkdir /content/checkpoints
!wget https://huggingface.co/stabilityai/stable-diffusion-2-1-base/resolve/main/v2-1_512-ema-pruned.ckpt -P /content/checkpoints

In [ ]:
# get stable diffusion v1.5 weight
# !mkdir /content/checkpoints
!wget https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned.ckpt -P /content/checkpoints

In [ ]:
!pip install gradio albumentations opencv-contrib-python imageio imageio-ffmpeg pytorch-lightning omegaconf test-tube streamlit einops transformers webdataset kornia open_clip_torch invisible-watermark streamlit-drawable-canvas torchmetrics timm addict yapf prettytable

In [ ]:
!pip install xformers triton

In [8]:
%cd /content/ControlNet

/content/ControlNet


In [9]:
!python tool_add_control.py /content/checkpoints/v1-5-pruned.ckpt /content/checkpoints/control_sd15_ini.ckpt

2023-02-17 02:49:51.741550: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-17 02:49:56.312348: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-17 02:49:56.312700: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-17 02:49:56.312724: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [ ]:
# tool_add_control.py for colab

"""
import sys
import os

assert len(sys.argv) == 3, 'Args are wrong.'

input_path = sys.argv[1]
output_path = sys.argv[2]

assert os.path.exists(input_path), 'Input model does not exist.'
assert not os.path.exists(output_path), 'Output filename already exists.'
assert os.path.exists(os.path.dirname(output_path)), 'Output path is not valid.'

import torch
from share import *
from cldm.model import create_model


def get_node_name(name, parent_name):
    if len(name) <= len(parent_name):
        return False, ''
    p = name[:len(parent_name)]
    if p != parent_name:
        return False, ''
    return True, name[len(parent_name):]


model = create_model(config_path='./models/cldm_v15.yaml')  # cpu

# pretrained_weights = torch.load(input_path)
pretrained_weights = torch.load(input_path, map_location=torch.device('cuda'))  # cuda
if 'state_dict' in pretrained_weights:
    pretrained_weights = pretrained_weights['state_dict']

scratch_dict = model.state_dict()

target_dict = {}
for k in scratch_dict.keys():
    is_control, name = get_node_name(k, 'control_')
    if is_control:
        copy_k = 'model.diffusion_' + name
    else:
        copy_k = k
    if copy_k in pretrained_weights:
        target_dict[k] = pretrained_weights[copy_k].clone()
    else:
        target_dict[k] = scratch_dict[k].clone()
        print(f'These weights are newly added: {k}')

model.load_state_dict(target_dict, strict=True)
torch.save(model.state_dict(), output_path)
print('Done.')
"""

In [ ]:
!python tool_add_control_sd21.py /content/checkpoints/v2-1_512-ema-pruned.ckpt /content/checkpoints/control_sd21_ini.ckpt

Traceback (most recent call last):
  File "tool_add_control_sd21.py", line 9, in <module>
    assert os.path.exists(input_path), 'Input model does not exist.'
AssertionError: Input model does not exist.


In [10]:
!pip install pytorch-lightning==1.9.1
# after pip please restart runtime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.8/825.8 KB 32.6 MB/s eta 0:00:00
  Attempting uninstall: pytorch-lightning
    Found existing installation: pytorch-lightning 1.9.2
    Uninstalling pytorch-lightning-1.9.2:
      Successfully uninstalled pytorch-lightning-1.9.2


In [1]:
import pytorch_lightning as pl

In [1]:
#@title training
%cd /content/ControlNet

/content/ControlNet


In [2]:
# create my dataset
import json
import cv2
import numpy as np

from torch.utils.data import Dataset


class MyDataset(Dataset):
    def __init__(self, number):
        self.number = number
        self.data = []
        with open(f'/content/fill{number}/prompt.json', 'rt') as f:
            for line in f:
                self.data.append(json.loads(line))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]

        source_filename = item['source']
        target_filename = item['target']
        prompt = item['prompt']

        

        source = cv2.imread(f'/content/fill{self.number}/' + source_filename)
        target = cv2.imread(f'/content/fill{self.number}/' + target_filename)

        # Do not forget that OpenCV read images in BGR order.
        source = cv2.cvtColor(source, cv2.COLOR_BGR2RGB)
        target = cv2.cvtColor(target, cv2.COLOR_BGR2RGB)

        # Normalize source images to [0, 1].
        source = source.astype(np.float32) / 255.0

        # Normalize target images to [-1, 1].
        target = (target.astype(np.float32) / 127.5) - 1.0

        return dict(jpg=target, txt=prompt, hint=source)

In [ ]:
# Please check https://github.com/lllyasviel/ControlNet/issues/84, for google colab error
# callback functionの中でon_train_batch_startを実装したが意味なさそう
# pytorch-lightring==1.5.0なら問題なく動作するかも？その場合はcondaを使ってpytorchのversion等を変更しないといけないかも

In [ ]:
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from cldm.logger import ImageLogger
from cldm.model import create_model, load_state_dict


# Configs
resume_path = '/content/checkpoints/control_sd15_ini.ckpt'
batch_size = 1
logger_freq = 300
learning_rate = 1e-5
sd_locked = True
only_mid_control = False
number = 100


# First use cpu to load models. Pytorch Lightning will automatically move it to GPUs.
model = create_model('./models/cldm_v15.yaml').cpu()
model.load_state_dict(load_state_dict(resume_path, location='cpu'))
model.learning_rate = learning_rate
model.sd_locked = sd_locked
model.only_mid_control = only_mid_control


# Misc
dataset = MyDataset(number)
dataloader = DataLoader(dataset, num_workers=1, batch_size=batch_size, shuffle=True)
logger = ImageLogger(batch_frequency=logger_freq)
# trainer = pl.Trainer(gpus=1, precision=32, callbacks=[logger])
trainer = pl.Trainer(gpus=1, precision=32, callbacks=[logger], accumulate_grad_batches=1)  # But this will be 4x slower


# Train!
trainer.fit(model, dataloader)

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/utilities/distributed.py:258: LightningDeprecationWarning: `pytorch_lightning.utilities.distributed.rank_zero_only` has been deprecated in v1.8.1 and will be removed in v2.0.0. You can import it from `pytorch_lightning.utilities` instead.
  rank_zero_deprecation(
Error caught was: module 'triton.language' has no attribute 'constexpr'


ControlLDM: Running in eps-prediction mode
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, context_

Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.3.mlp.fc2.weight', 'vision_model.encoder.layers.12.self_attn.k_proj.bias', 'vision_model.encoder.layers.13.layer_norm1.bias', 'vision_model.encoder.layers.16.layer_norm1.bias', 'vision_model.encoder.layers.7.self_attn.v_proj.bias', 'vision_model.encoder.layers.13.layer_norm1.weight', 'vision_model.encoder.layers.1.layer_norm1.bias', 'vision_model.encoder.layers.5.layer_norm2.bias', 'vision_model.encoder.layers.23.self_attn.k_proj.bias', 'vision_model.encoder.layers.16.layer_norm2.bias', 'vision_model.encoder.layers.12.mlp.fc1.bias', 'vision_model.encoder.layers.18.layer_norm1.bias', 'vision_model.encoder.layers.15.layer_norm2.weight', 'vision_model.encoder.layers.16.self_attn.v_proj.weight', 'vision_model.encoder.layers.1.self_attn.k_proj.weight', 'vision_model.encoder.layers.4.layer_norm1.weight', 'vision_model.encoder.layers.18.mlp.fc1.wei

Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, context_dim is 768 and using 8 heads.
Setting up Me

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:466: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/configuration_validator.py:108: PossibleUserWarning: You defined a `validation_step` but

Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/utilities/data.py:84: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 1. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:232: UserWarning: You called `self.log('global_step', ...)` in your `training_step` but the value needs to be floating point. Converting it to torch.float32.
  warning_cache.warn(
